# My preferred MultiIndex DataFrame structure for stock market data

## Configuring your development environment

In [2]:
# install the required python packages
# !pip install numpy pandas yfinance

## Configuring our date range and ticker

In [1]:
# import the necessary packages
from datetime import timedelta
from datetime import datetime
import yfinance as yf
import pandas as pd

# set the start and end dates for our market data
end_date = datetime(year=2025, month=3, day=1)
start_date = end_date - timedelta(days=365)

# set the name of the ticker we want to download market data for
ticker = ["NVDA", "AAPL"]

## Understanding yfinance's default multi-index structure

In [2]:
# download daily market data
df_orig = yf.download(
    tickers=ticker,
    start=start_date,
    end=end_date,
    interval="1d",
    auto_adjust=True,
    progress=False
)
df_orig

Price            Close                    High                     Low  \
Ticker            AAPL        NVDA        AAPL        NVDA        AAPL   
Date                                                                     
2024-03-01  178.815674   82.248108  179.681580   82.269104  176.546390   
2024-03-04  174.277115   85.205002  176.068644   87.662079  172.973259   
2024-03-05  169.320496   85.935760  171.231471   86.068720  168.822845   
2024-03-06  168.325180   88.670860  170.435227   89.694526  167.887245   
2024-03-07  168.205750   92.638550  169.927615   92.736517  167.698152   
...                ...         ...         ...         ...         ...   
2025-02-24  247.100006  130.268021  248.860001  138.577254  244.419998   
2025-02-25  247.039993  126.618355  250.000000  130.188026  244.910004   
2025-02-26  240.360001  131.267929  244.979996  133.717701  239.130005   
2025-02-27  237.300003  120.138954  242.460007  134.997581  237.059998   
2025-02-28  241.839996  124.908508  242.089996  125.078491  230.199997   

Price                         Open                Volume             
Ticker            NVDA        AAPL        NVDA      AAPL       NVDA  
Date                                                                 
2024-03-01   79.405174  178.706190   79.969964  73488000  479135000  
2024-03-04   83.687574  175.322168   84.098415  81510100  615616000  
2024-03-05   83.389600  169.957487   85.241988  95132400  520639000  
2024-03-06   87.001411  170.256065   87.993089  68587700  582520000  
2024-03-07   89.572556  168.355038   90.128373  71765100  608119000  
...                ...         ...         ...       ...        ...  
2025-02-24  130.068042  244.929993  136.547442  51326400  251381100  
2025-02-25  124.428561  248.000000  129.968045  48013300  271428700  
2025-02-26  128.478192  244.330002  129.978054  44433600  322553800  
2025-02-27  119.998968  239.410004  134.987587  41153600  443175800  
2025-02-28  116.389295  236.949997  118.009141  56833400  389091100  

[250 rows x 10 columns]

In [3]:
# show the original index structure
df_orig.index

DatetimeIndex(['2024-03-01', '2024-03-04', '2024-03-05', '2024-03-06',
               '2024-03-07', '2024-03-08', '2024-03-11', '2024-03-12',
               '2024-03-13', '2024-03-14',
               ...
               '2025-02-14', '2025-02-18', '2025-02-19', '2025-02-20',
               '2025-02-21', '2025-02-24', '2025-02-25', '2025-02-26',
               '2025-02-27', '2025-02-28'],
              dtype='datetime64[ns]', name='Date', length=250, freq=None)

In [4]:
# show the original columns
df_orig.columns

MultiIndex([( 'Close', 'AAPL'),
            ( 'Close', 'NVDA'),
            (  'High', 'AAPL'),
            (  'High', 'NVDA'),
            (   'Low', 'AAPL'),
            (   'Low', 'NVDA'),
            (  'Open', 'AAPL'),
            (  'Open', 'NVDA'),
            ('Volume', 'AAPL'),
            ('Volume', 'NVDA')],
           names=['Price', 'Ticker'])

In [5]:
df_orig["Close"]

Ticker,AAPL,NVDA
Date,,
2024-03-01,178.815674,82.248108
2024-03-04,174.277115,85.205002
2024-03-05,169.320496,85.935760
2024-03-06,168.325180,88.670860
2024-03-07,168.205750,92.638550
...,...,...
2025-02-24,247.100006,130.268021
2025-02-25,247.039993,126.618355
2025-02-26,240.360001,131.267929


In [6]:
df_orig["Close"]["AAPL"]

Date
2024-03-01    178.815674
2024-03-04    174.277115
2024-03-05    169.320496
2024-03-06    168.325180
2024-03-07    168.205750
                 ...    
2025-02-24    247.100006
2025-02-25    247.039993
2025-02-26    240.360001
2025-02-27    237.300003
2025-02-28    241.839996
Name: AAPL, Length: 250, dtype: float64

## Updating yfinance's multi-index structure to use row indexes

In [7]:
# clone the original dataframe
df = df_orig.copy()

# restructure the default multi-index yfinance dataframe by converting from
# wide to long format, renaming the indices, ensuring the columns are provided
# in OHLCV order, reordering the index such that date is first and symbol is
# second, and finally sorting the index itself
df = df.stack(level="Ticker", future_stack=True)
df.index.names = ["Date", "Symbol"]
df = df[["Open", "High", "Low", "Close", "Volume"]]
df = df.swaplevel(0, 1)
df = df.sort_index()
df

Price                    Open        High         Low       Close     Volume
Symbol Date                                                                 
AAPL   2024-03-01  178.706190  179.681580  176.546390  178.815674   73488000
       2024-03-04  175.322168  176.068644  172.973259  174.277115   81510100
       2024-03-05  169.957487  171.231471  168.822845  169.320496   95132400
       2024-03-06  170.256065  170.435227  167.887245  168.325180   68587700
       2024-03-07  168.355038  169.927615  167.698152  168.205750   71765100
...                       ...         ...         ...         ...        ...
NVDA   2025-02-24  136.547442  138.577254  130.068042  130.268021  251381100
       2025-02-25  129.968045  130.188026  124.428561  126.618355  271428700
       2025-02-26  129.978054  133.717701  128.478192  131.267929  322553800
       2025-02-27  134.987587  134.997581  119.998968  120.138954  443175800
       2025-02-28  118.009141  125.078491  116.389295  124.908508  389091100

[500 rows x 5 columns]

In [8]:
# show the updated index structure
df.index

MultiIndex([('AAPL', '2024-03-01'),
            ('AAPL', '2024-03-04'),
            ('AAPL', '2024-03-05'),
            ('AAPL', '2024-03-06'),
            ('AAPL', '2024-03-07'),
            ('AAPL', '2024-03-08'),
            ('AAPL', '2024-03-11'),
            ('AAPL', '2024-03-12'),
            ('AAPL', '2024-03-13'),
            ('AAPL', '2024-03-14'),
            ...
            ('NVDA', '2025-02-14'),
            ('NVDA', '2025-02-18'),
            ('NVDA', '2025-02-19'),
            ('NVDA', '2025-02-20'),
            ('NVDA', '2025-02-21'),
            ('NVDA', '2025-02-24'),
            ('NVDA', '2025-02-25'),
            ('NVDA', '2025-02-26'),
            ('NVDA', '2025-02-27'),
            ('NVDA', '2025-02-28')],
           names=['Symbol', 'Date'], length=500)

In [9]:
# as well as the updated columns
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object', name='Price')

In [10]:
# display *just* the subset of AAPL market data
df.xs("AAPL")

Price,Open,High,Low,Close,Volume
Date,,,,,
2024-03-01,178.706190,179.681580,176.546390,178.815674,73488000
2024-03-04,175.322168,176.068644,172.973259,174.277115,81510100
2024-03-05,169.957487,171.231471,168.822845,169.320496,95132400
2024-03-06,170.256065,170.435227,167.887245,168.325180,68587700
2024-03-07,168.355038,169.927615,167.698152,168.205750,71765100
...,...,...,...,...,...
2025-02-24,244.929993,248.860001,244.419998,247.100006,51326400
2025-02-25,248.000000,250.000000,244.910004,247.039993,48013300
2025-02-26,244.330002,244.979996,239.130005,240.360001,44433600


## Performing data analysis operations on our new multi-index DataFrame

In [11]:
# notice how easy it is to analyze OHLCV data for each of the symbols with
# this structure by (1) first grouping on the symbol, followed by (2) applying
# our analysis method via 'transform'
df["50MA"] = df.groupby(level="Symbol")["Close"].transform(
    lambda x: x.rolling(window=50).mean()
)
df

Price                    Open        High         Low       Close     Volume  \
Symbol Date                                                                    
AAPL   2024-03-01  178.706190  179.681580  176.546390  178.815674   73488000   
       2024-03-04  175.322168  176.068644  172.973259  174.277115   81510100   
       2024-03-05  169.957487  171.231471  168.822845  169.320496   95132400   
       2024-03-06  170.256065  170.435227  167.887245  168.325180   68587700   
       2024-03-07  168.355038  169.927615  167.698152  168.205750   71765100   
...                       ...         ...         ...         ...        ...   
NVDA   2025-02-24  136.547442  138.577254  130.068042  130.268021  251381100   
       2025-02-25  129.968045  130.188026  124.428561  126.618355  271428700   
       2025-02-26  129.978054  133.717701  128.478192  131.267929  322553800   
       2025-02-27  134.987587  134.997581  119.998968  120.138954  443175800   
       2025-02-28  118.009141  125.078491  116.389295  124.908508  389091100   

Price                    50MA  
Symbol Date                    
AAPL   2024-03-01         NaN  
       2024-03-04         NaN  
       2024-03-05         NaN  
       2024-03-06         NaN  
       2024-03-07         NaN  
...                       ...  
NVDA   2025-02-24  134.377040  
       2025-02-25  134.208256  
       2025-02-26  134.047671  
       2025-02-27  133.703902  
       2025-02-28  133.517319  

[500 rows x 6 columns]

In [12]:
# let's investigate the computed 50MA for AAPL
df.xs("AAPL")["50MA"]

Date
2024-03-01           NaN
2024-03-04           NaN
2024-03-05           NaN
2024-03-06           NaN
2024-03-07           NaN
                 ...    
2025-02-24    240.485810
2025-02-25    240.476652
2025-02-26    240.359467
2025-02-27    240.151713
2025-02-28    240.031363
Name: 50MA, Length: 250, dtype: float64

In [13]:
# now, let's *manually* compute the 50MA for AAPL (i.e., on a series rather
# than a multi-index dataframe)
df.xs("AAPL")["Close"].rolling(window=50).mean()

Date
2024-03-01           NaN
2024-03-04           NaN
2024-03-05           NaN
2024-03-06           NaN
2024-03-07           NaN
                 ...    
2025-02-24    240.485810
2025-02-25    240.476652
2025-02-26    240.359467
2025-02-27    240.151713
2025-02-28    240.031363
Name: Close, Length: 250, dtype: float64

In [14]:
# verify our calculations match
pd.testing.assert_series_equal(
    df.xs("AAPL")["50MA"],
    df.xs("AAPL")["Close"].rolling(window=50).mean(),
    check_names=False
)